In [102]:
import pandas as pd
import numpy as np
from scipy.stats import poisson 
from matplotlib import pyplot as plt
import seaborn as sns


In [103]:
tabla = pd.read_html('https://mexico.as.com/resultados/futbol/mexico_apertura/clasificacion/',encoding='UTF-8')
club = tabla[1]['Equipo'].str.split("  ", n = 1, expand = True)
club1 = club[1].str.split("  ", n = 1, expand = True)
#club1[0][3]='Xolos'
club=club1[0]
jornada = pd.read_html('https://mexico.as.com/resultados/futbol/mexico_apertura/calendario/?omnil=mpal')
tablalocal = tabla[1]
tablavisita = tabla[2]
tablagral=tablalocal+tablavisita
tablagral.Equipo = club
stats_gral=tablagral.describe()
jornadascompletas=int()
local=jornada[jornadascompletas]['Equipo local']
visita=jornada[jornadascompletas]['Equipo visitante']
club

0                  Rayados
1                   Toluca
2                   Tigres
3                  Pachuca
4                   Necaxa
5                   Puebla
6                    Xolos
7            Santos Laguna
8                  América
9     Mazatlán Fútbol Club
10                 León FC
11                   Atlas
12                   Pumas
13       Atlético San Luis
14               Cruz Azul
15                  Bravos
16                  Chivas
17          Gallos Blancos
Name: 0, dtype: object

In [104]:
GF_promedio=tablagral.GF/tablagral.PJ
GC_promedio=tablagral.GC/tablagral.PJ
goles_promedio_ligamx=stats_gral.GF[1]/stats_gral.PJ[1]
ataque=GF_promedio/goles_promedio_ligamx
defensa=GC_promedio/goles_promedio_ligamx
mx_av=pd.DataFrame({'Equipo':club, 'GF prom':GF_promedio,'ataque':ataque,'GC prom':GC_promedio,'defensa':defensa})
matrix_goles=pd.DataFrame()
ind=list(range(0,len(mx_av)))

for i in ind:
    matrix_goles[i]=mx_av['ataque'][ind[i]]*mx_av['defensa'][:]*goles_promedio_ligamx

matrix_goles.columns=club
matrix_goles.index=club
GFx_local=np.diag(matrix_goles[local].loc[visita])
GFx_visita=np.diag(matrix_goles[visita].loc[local])

k=[0,1,2,3,4,5]
P_home=pd.DataFrame()
P_away=pd.DataFrame()
mu=GFx_local
mu2=GFx_visita

for i in k:
    P_home[i]=poisson.pmf(k[i],mu)
    P_away[i]=poisson.pmf(k[i],mu2)

P_home.index=local
P_away.index=visita

tamañojornada=len(local)
g=pd.DataFrame()
ind=list(range(0,tamañojornada))

for i in ind: 
    tabla=pd.DataFrame({local[i]:P_home.iloc[i].tolist(),visita[i]:P_away.iloc[i].tolist()})
    tabla.to_csv('tablamx{}.csv'.format(i),index=False, sep=',')
    p=tabla.idxmax()
    g=pd.concat([g,p])

g=g[0]
Pgoles_home=g[local]
Pgoles_away=g[visita]
Pgoles_home.index=ind
Pgoles_away.index=ind

tabla=pd.DataFrame()
winlcl=pd.DataFrame()
winvis=pd.DataFrame()
e=pd.DataFrame()
ind=list(range(0,9))

for i in ind:
    tabla=pd.read_csv('tablamx{}.csv'.format(ind[i]))
    empate=pd.Series(tabla.iloc[0][0]*tabla.iloc[0][1]+tabla.iloc[1][0]*tabla.iloc[1][1]+tabla.iloc[2][0]*tabla.iloc[2][1]+tabla.iloc[3][0]*tabla.iloc[3][1]+tabla.iloc[4][0]*tabla.iloc[4][1]+tabla.iloc[5][0]*tabla.iloc[5][1])
    vis=pd.Series(tabla.iloc[0][0]*tabla.iloc[1][1]+tabla.iloc[0][0]*tabla.iloc[2][1]+tabla.iloc[0][0]*tabla.iloc[3][1]+tabla.iloc[0][0]*tabla.iloc[4][1]+tabla.iloc[0][0]*tabla.iloc[5][1]+tabla.iloc[1][0]*tabla.iloc[2][1]+tabla.iloc[1][0]*tabla.iloc[3][1]+tabla.iloc[1][0]*tabla.iloc[4][1]+tabla.iloc[1][0]*tabla.iloc[5][1]+tabla.iloc[2][0]*tabla.iloc[3][1]+tabla.iloc[2][0]*tabla.iloc[4][1]+tabla.iloc[2][0]*tabla.iloc[5][1]+tabla.iloc[3][0]*tabla.iloc[4][1]+tabla.iloc[3][0]*tabla.iloc[5][1]+tabla.iloc[4][0]*tabla.iloc[5][1])
    lcl=pd.Series(tabla.iloc[0][1]*tabla.iloc[1][0]+tabla.iloc[0][1]*tabla.iloc[2][0]+tabla.iloc[0][1]*tabla.iloc[3][0]+tabla.iloc[0][1]*tabla.iloc[4][0]+tabla.iloc[0][1]*tabla.iloc[5][0]+tabla.iloc[1][1]*tabla.iloc[2][0]+tabla.iloc[1][1]*tabla.iloc[3][0]+tabla.iloc[1][1]*tabla.iloc[4][0]+tabla.iloc[1][1]*tabla.iloc[5][0]+tabla.iloc[2][1]*tabla.iloc[3][0]+tabla.iloc[2][1]*tabla.iloc[4][0]+tabla.iloc[2][1]*tabla.iloc[5][0]+tabla.iloc[3][1]*tabla.iloc[4][0]+tabla.iloc[3][1]*tabla.iloc[5][0]+tabla.iloc[4][1]*tabla.iloc[5][0])
    e=pd.concat([e,empate])
    winvis=pd.concat([winvis,vis])
    winlcl=pd.concat([winlcl,lcl])

winvis.index=ind
winlcl.index=ind
e.index=ind
visita_gane=winvis
local_gane=winlcl
empate=e

esperanza = pd.DataFrame({'local':local,'E GF local':GFx_local,'E GF visita':GFx_visita,'visita':visita})
esperanza.to_csv('mx_esperanza_goles_jornada{}.csv'.format(jornadascompletas),index=False, sep=',')
esperanza

,local,E GF local,E GF visita,visita
0,Atlas,0.877544,0.902616,Bravos
1,Mazatlán Fútbol Club,2.005814,0.965298,Gallos Blancos
2,Puebla,1.353924,1.128270,Necaxa
3,Toluca,1.579578,1.905523,Rayados
4,Cruz Azul,1.418397,2.048796,Xolos
5,Pachuca,1.031561,0.654960,América
6,Santos Laguna,2.941860,1.170058,León FC
7,Atlético San Luis,0.687708,0.713178,Pumas
8,Chivas,0.429817,0.902616,Tigres


In [105]:
pronostico = pd.DataFrame({'local':local,'P GF local':Pgoles_home,'P GF visita':Pgoles_away,'visita':visita})
pronostico.to_csv('mx_pronostico_goles_jornada{}.csv'.format(jornadascompletas),index=False, sep=',')
pronostico

,local,P GF local,P GF visita,visita
0,Atlas,0.0,0.0,Bravos
1,Mazatlán Fútbol Club,2.0,0.0,Gallos Blancos
2,Puebla,1.0,1.0,Necaxa
3,Toluca,1.0,1.0,Rayados
4,Cruz Azul,1.0,2.0,Xolos
5,Pachuca,1.0,0.0,América
6,Santos Laguna,2.0,1.0,León FC
7,Atlético San Luis,0.0,0.0,Pumas
8,Chivas,0.0,0.0,Tigres


In [106]:
probabilidades=pd.DataFrame({'local':local, 'P ganar local':local_gane[0], 'P empate':empate[0], 'P ganar visita':visita_gane[0], 'visita':visita})
probabilidades.sort_values(by='P ganar local', ascending=False)
probabilidades.to_csv('mx_probabilidades_jornada{}.csv'.format(jornadascompletas),index=False, sep=',')
probabilidades

,local,P ganar local,P empate,P ganar visita,visita
0,Atlas,0.327288,0.331059,0.341004,Bravos
1,Mazatlán Fútbol Club,0.598466,0.210179,0.174094,Gallos Blancos
2,Puebla,0.416572,0.269064,0.310545,Necaxa
3,Toluca,0.317402,0.220438,0.443159,Rayados
4,Cruz Azul,0.261908,0.213349,0.503007,Xolos
5,Pachuca,0.437586,0.333050,0.228604,América
6,Santos Laguna,0.656080,0.144348,0.120183,León FC
7,Atlético San Luis,0.300826,0.382866,0.316126,Pumas
8,Chivas,0.168826,0.376558,0.454261,Tigres


In [107]:
j=list(range(0,jornadascompletas))
resultado_local=pd.DataFrame()
resultado_visita=pd.DataFrame()

for i in j:
        resultado=jornada[i].Resultado.str.split("-", n = 1, expand = True).astype(int)
        resultado_local[i]=pd.Series(resultado[0])
        resultado_visita[i]=pd.Series(resultado[1])

resultado_local=resultado_local.fillna(0)
resultado_visita=resultado_visita.fillna(0)
resultado_local.to_csv('mx_resultado_local.csv')
resultado_visita.to_csv('mx_resultado_visita.csv')
equipo_local=pd.DataFrame()
equipo_visita=pd.DataFrame()

for i in j:
    equipo_local[i]=jornada[i]['Equipo local']
    equipo_visita[i]=jornada[i]['Equipo visitante']

equipo_local=equipo_local.fillna(0)
equipo_visita=equipo_visita.fillna(0)
equipo_local.to_csv('mx_equipo_local.cvs',sep=',')
equipo_visita.to_csv('mx_equipo_visita.cvs',sep=',')

for i in j:
    pd.DataFrame({'equipo local':equipo_local[i],'goles local':resultado_local[i],'goles visita':resultado_visita[i],'equipo visita':equipo_visita[i]}).to_csv('mx_jornada{}.csv'.format(i),index=False)

suma_goles_local_jornada=resultado_local.sum()
suma_goles_visita_jornada=resultado_visita.sum()
dif_goles=suma_goles_local_jornada-suma_goles_visita_jornada

ValueError: invalid literal for int() with base 10: 'X 21:05'

In [ ]:
sns.kdeplot(data=suma_goles_local_jornada,shade=True,bw_adjust=1,label='local',alpha=0.6)
sns.kdeplot(data=suma_goles_visita_jornada,shade=True,bw_adjust=1,label='visitante',alpha=0.6)
plt.xlabel("Conteo de goles hasta la jornada {} ".format(jornadascompletas))
plt.legend(loc='upper left', prop={'size': 10}, frameon=False)
plt.title('Densidad de probabilidad de total de goles liga MX 2022-2023')
plt.savefig('mx_dist_local_visita.png')
plt.show()

In [ ]:
sns.distplot(dif_goles)
plt.xlabel("Conteo de la differencia de goles hasta la jornada {} ".format(jornadascompletas))
plt.title('Liga MX 2022-2023')
plt.savefig('mx_distribucion_dif.png')
plt.show()

In [ ]:
plt.plot(dif_goles,"--X",markersize=12)
plt.bar(j,dif_goles,alpha=.5)
plt.xlabel("Jornada")
plt.ylabel("Total de Diferencia de goles")
plt.title('Liga mx 2022-2023')
plt.xticks([0,1,2,3])
plt.yticks([-3,0,3])
plt.savefig('mx_diffbarras.png')
plt.show()

In [ ]:
y=list(tablagral.GF-tablagral.GC)
x=list(range(0,len(club)))
X=list(range(0,max(y)))
plt.plot(club,y,'s--')
plt.xticks(x, club, rotation='vertical',fontsize=12)
plt.ylabel('Diferencia goles',fontsize=12)
plt.title("Liga MX apertura 2022-2023",fontsize=12)
plt.savefig('mx_dif_equipos.png')
plt.show()


In [ ]:
local_cumulativo=np.array(suma_goles_local_jornada)
visita_cumulativo=np.array(suma_goles_visita_jornada)
plt.plot(np.cumsum(local_cumulativo),"--X",markersize=12,label='local',alpha=0.8,linewidth=1.5)
plt.plot(np.cumsum(visita_cumulativo),"--X",markersize=12,label='visita',alpha=0.8,linewidth=1.5)
plt.plot(np.cumsum(local_cumulativo-visita_cumulativo),"--X",markersize=12,label='diferencia',alpha=0.8,linewidth=1.5)
plt.plot(np.cumsum(local_cumulativo+visita_cumulativo),"--X",markersize=12,label='suma',alpha=0.8,linewidth=1.5)
plt.legend(loc='upper left', prop={'size': 10}, frameon=False)
plt.title("Cumulativo de goles Liga MX 2022-2023")
plt.ylabel('Goles')
plt.xlabel('Jornada')
plt.xticks([0,1,2,3])
plt.savefig('mx_cumulativo.png')
plt.show()

In [ ]:
jornada